In [1]:
import requests
import pandas as pd
import json
from psaw import PushshiftAPI
from nltk import ngrams
from bs4 import *
api = PushshiftAPI()

In [107]:
# function that gets comments dataframe for a subreddit

def get_pushshift_data(data_type, **kwargs):

 
    base_url = f"https://api.pushshift.io/reddit/search/{data_type}/"
    payload = kwargs
    request = requests.get(base_url, params=payload)
    data = request.json().get("data")
    df = pd.DataFrame.from_records(data)[['author','subreddit','body','created_utc']]
    return df

#  function that counts the number of n-grams in a column of a reddit dataframe

def count_grams(df, i):
    df_body = df['body']
    
    grams_dict = {}
    for s in df_body:
        grams = ngrams(s.split(),i)

        for gram in grams:
            gram2 = ' '.join(gram)
            if gram2 in grams_dict:
                grams_dict[gram2] += 1
            else:
                grams_dict[gram2] = 1

    final_df = pd.DataFrame([grams_dict]).T

    return final_df

# collects gram counts from multiple subreddits
def collect_grams(subs,i):
    final_df = pd.DataFrame()
    error_list = []
    for s in subs:
        try:
            df1 = get_pushshift_data(data_type='comment',subreddit=s, after="30d", size = 500)
            count_df = count_grams(df1,i)
            count_df[str(i)+'-gram'] = count_df.index
            count_df['subreddit'] = s
            count_df = count_df.rename(columns={0:'count'})
            final_df = pd.concat([count_df, final_df], ignore_index=True)
        except:
            error_list.append(s)
        
    error = len(error_list)/len(subs)
    print(error)
    
    return final_df, error_list

In [88]:
subs_df = pd.read_csv('list_of_subreddits.csv')
subs_list = [s[3:] for s in subs_df['Subreddits']]

subs_list
subs_100 = subs_list[:100]

In [ ]:
collect_grams(subs_100,1)